In [1]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('VarunMYSQL').config('spark.jars.packages','mysql:mysql-connector-java:8.0.28').master('yarn').getOrCreate()

In [ ]:
employees = spark.read.format('jdbc').option('url','jdbc:mysql://master:3306/varunm15t38hedu')\
        .option('user','varunm15t38hedu')\
        .option('password','PlumWaspbca52@')\
        .option('dbtable','employees')\
        .option('driver','com.mysql.cj.jdbc.Driver').load()

In [4]:
employees.show()

+-----------+-----------+----------+--------+------------+----------+----------+--------+--------------+----------+-------------+
|employee_id| first_name| last_name|   email|phone_number| hire_date|    job_id|  salary|commission_pct|manager_id|department_id|
+-----------+-----------+----------+--------+------------+----------+----------+--------+--------------+----------+-------------+
|        100|     Steven|      King|   SKING|515.123.4567|1987-06-17|   AD_PRES|24000.00|          null|      null|           90|
|        101|      Neena|   Kochhar|NKOCHHAR|515.123.4568|1989-09-21|     AD_VP|17000.00|          null|       100|           90|
|        102|        Lex|   De Haan| LDEHAAN|515.123.4569|1993-01-13|     AD_VP|17000.00|          null|       100|           90|
|        103|  Alexander|    Hunold| AHUNOLD|590.423.4567|1990-01-03|   IT_PROG| 9000.00|          null|       102|           60|
|        104|      Bruce|     Ernst|  BERNST|590.423.4568|1991-05-21|   IT_PROG| 6000.00| 

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [6]:
sch = StructType([ StructField('department_id',IntegerType(),False),StructField('dname',StringType(),False),StructField('manager_id',IntegerType(),False),StructField('location_id',IntegerType(),False)])

In [7]:
departments = spark.readStream.format('socket').option('host','localhost').option('port',9940).load()

25/06/16 12:26:08 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [8]:
deptdata=departments.select(from_json(col('value').cast('string'),sch).alias('value'))

In [9]:
deptdata.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- department_id: integer (nullable = true)
 |    |-- dname: string (nullable = true)
 |    |-- manager_id: integer (nullable = true)
 |    |-- location_id: integer (nullable = true)



### Joining batch vs stream data

In [10]:
tradeDF=deptdata.select("value.*")

In [11]:
j1=employees.join(tradeDF,employees.department_id==tradeDF.department_id,'inner')

In [ ]:
j1.writeStream.format('console').outputMode('update').option('checkpointLocation','chk134').trigger(processingTime='1 minute').start().awaitTermination()